In [1]:
!pip install implicit
!pip install fastparquet

     |████████████████████████████████| 1.0 MB 9.3 MB/s eta 0:00:01
     |████████████████████████████████| 777 kB 8.5 MB/s eta 0:00:01
     |████████████████████████████████| 2.7 MB 10.2 MB/s eta 0:00:01


In [2]:
import pandas as pd
import numpy as np
import scipy.sparse as sparse
import random
import implicit
import datetime

from tqdm import tqdm

In [3]:
#try:
#  path = 'C:/Users/User/Desktop/AIB_13/CP2/data/'
#  df = pd.read_parquet(path + 'light_2019-Oct.parquet', engine='fastparquet')
#except:
#  path = '/content/drive/MyDrive/CP2/data/'
#  df = pd.read_parquet(path + 'light_2019-Oct.parquet', engine='fastparquet')

df = pd.read_parquet('light_2019-Oct.parquet', engine='fastparquet')

In [4]:
df['event_time'] = df['event_time'].apply(lambda x : x[:-4]).astype('datetime64')
df['event_time'].dtype

dtype('<M8[ns]')

In [5]:
df['event_time'] = df['event_time'] + datetime.timedelta(hours=4)
df['event_time'].tail()

42448759   2019-11-01 03:59:58
42448760   2019-11-01 03:59:58
42448761   2019-11-01 03:59:58
42448762   2019-11-01 03:59:59
42448763   2019-11-01 03:59:59
Name: event_time, dtype: datetime64[ns]

In [6]:
df.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01 04:00:00,view,44600062,-251657396,NaN,shiseido,35.790001,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
1,2019-10-01 04:00:00,view,3900821,-780140327,appliances.environment.water_heater,aqua,33.200001,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2,2019-10-01 04:00:01,view,17200506,-1904213353,furniture.living_room.sofa,NaN,543.099976,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8
3,2019-10-01 04:00:01,view,1307067,1518338663,computers.notebook,lenovo,251.740005,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713
4,2019-10-01 04:00:04,view,1004237,-1769995873,electronics.smartphone,apple,1081.979980,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d


In [7]:
drop_user_id = df.loc[df['event_type'] != 'view', 'user_id']

In [8]:
df = df.loc[~df['user_id'].isin(drop_user_id)].reset_index()

df.head()

,index,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,1,2019-10-01 04:00:00,view,3900821,-780140327,appliances.environment.water_heater,aqua,33.200001,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
1,2,2019-10-01 04:00:01,view,17200506,-1904213353,furniture.living_room.sofa,NaN,543.099976,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8
2,3,2019-10-01 04:00:01,view,1307067,1518338663,computers.notebook,lenovo,251.740005,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713
3,4,2019-10-01 04:00:04,view,1004237,-1769995873,electronics.smartphone,apple,1081.979980,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d
4,6,2019-10-01 04:00:08,view,17300353,746586423,NaN,creed,380.959991,555447699,4fe811e9-91de-46da-90c3-bbd87ed3a65d


In [9]:
data = df[['user_id','user_session','product_id','event_time']]

In [10]:
data.set_index(['user_id','user_session','product_id'])

,,,event_time
user_id,user_session,product_id,
554748717,9333dfbd-b87a-4708-9857-6336556b0fcc,3900821,2019-10-01 04:00:00
519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8,17200506,2019-10-01 04:00:01
550050854,7c90fc70-0e80-4590-96f3-13c02c18c713,1307067,2019-10-01 04:00:01
535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d,1004237,2019-10-01 04:00:04
555447699,4fe811e9-91de-46da-90c3-bbd87ed3a65d,17300353,2019-10-01 04:00:08
...,...,...,...
537931532,22c57267-da98-4f28-9a9c-18bb5b385193,2300275,2019-11-01 03:59:58
527322328,5054190a-46cb-4211-a8f1-16fc1a060ed8,10800172,2019-11-01 03:59:58
566280422,05b6c62b-992f-4e8e-91f7-961bcb4719cd,5701038,2019-11-01 03:59:58


In [11]:
def func(user_id):
  results = []
  temp = data.loc[data['user_id'] == user_id]
  user_sessions = temp['user_session'].unique()
  for user_session in user_sessions:
    temp2 = temp.loc[temp['user_session'] == user_session]
    product_ids = temp2['product_id'].unique()
    for product_id in product_ids:
      temp3 = temp2.loc[temp2['product_id'] == product_id]
     
      #display(temp3)
      if temp3.shape[0] == 1:
        return [user_id, product_id, None]
      else:
        
        #display(temp3['event_time'].diff().sum() / (temp3['event_time'].diff().shape[0]-1))
        result = temp3['event_time'].diff().sum() / (temp3['event_time'].diff().shape[0]-1)
        results.append([user_id, product_id, result])
  return results

In [12]:
user_id = 184265397
lst = func(user_id)
for value in lst:
  if type(value) == list:
    print(value)
  else:
    print(lst)
    break

[184265397, 6902133, Timedelta('0 days 00:00:41')]
[184265397, 6902303, Timedelta('0 days 00:00:14')]
[184265397, 27400002, Timedelta('0 days 00:00:29')]


In [13]:
user_id = 33869381
func(user_id)

[33869381, 7002639, None]

In [14]:
user_id = 566243205
func(user_id)

[566243205, 1004739, None]

In [15]:
user_id = 554006311
func(user_id)

[[554006311, 4802036, Timedelta('0 days 12:13:28.816666666')]]

In [16]:
user_ids = df['user_id'].unique()

In [ ]:
frame = []
for user_id in tqdm(user_ids):
  lst = func(user_id)

  for value in lst:
    if type(value) == list:
      frame.append(value)
    else:
      frame.append(lst)
      break

  3%|█                               | 82799/2540832 [09:42<4:41:32, 145.51it/s]

In [18]:
result_frame = pd.DataFrame(frame, columns=['user_id','product_id','term'])

In [19]:
result_frame.to_parquet('add_term.parquet')

Timedelta('0 days 00:01:47')